# Slab generation by pymatgen

In [7]:
# Uncomment the next line to install pymatgen
#!pip install pymatgen

In [8]:
# Import MPRester to query MaterialsProject API
from pymatgen.ext.matproj import MPRester
# Import SlabGenerator to generate slabs
from pymatgen.core.surface import SlabGenerator

**Here, we define the parameters of our slabs**

In [9]:
# Structure ID to query
mpid = 'mp-81'  # FCC Au

# Miller index
miller_index = (1, 1, 1)

# Whether slab and vacuum thicknesses are in number of layers or Angstroms
in_unit_planes = False

# Slab and vacuum thicknesses in Angstroms
slab_thickness = 10
vacuum_thickness = 20

# Whether to generate symmetric slabs
symmetrize = True

**and proceed with the structure querying and slab generation**

In [10]:
# Then, we first query the conventional cell of our structure
with MPRester() as m:
    conv_bulk = m.get_structure_by_material_id(mpid, conventional_unit_cell=True)

# Then, initialize the SlabGenerator object
sg = SlabGenerator(initial_structure=conv_bulk,  # conventional bulk structure
                   miller_index=miller_index,  # miller index for slabs
                   min_slab_size=slab_thickness,  # minimum slab thickness
                   min_vacuum_size=vacuum_thickness,  # minimum vacuum region thickness
                   center_slab=True,  # whether to center the slabs in the c-direction
                   in_unit_planes=in_unit_planes,
                   primitive=True,  # whether to apply cell reduction on generated slabs
                   max_normal_search=max([abs(m) for m in miller_index]))

# and generate slabs
slabs = sg.get_slabs(symmetrize=symmetrize)

# We can save the slabs as POSCARs
miller_to_str = ''.join([str(m) for m in miller_index])
for index, slab in enumerate(slabs): #  each slab in slabs correspond to a different termination
    formula = slab.composition.reduced_formula
    slab.to('poscar', f'{formula}_{miller_to_str}_{index}.vasp')